In [ ]:
import dolfin as d
import smart
import h5py
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import tkinter as tk
import os, sys
import re

sys.path.append("/root/shared/gitrepos/smart-comp-sci/utils")
import spread_cell_mesh_generation as mesh_gen

from tkinter import filedialog
from tkinter import messagebox
root = tk.Tk()
root.withdraw()

parent_path = "/root/scratch/smart-comp-sci-data/mechanotransduction/results_nanopillars"
mesh_folder = "/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars"
folder_list = os.listdir(parent_path)
ind_files = False
display = False
axisymm = False
symm = 0.125
# outerExpr13 = "(1 - z**4/(2000+z**4)) * (r**2 + z**2) + 0.4*(r**2 + (z+9.72)**2)*z**4 / (15 + z**4) - 169"
# innerExpr13 = "(r/5.3)**2 + ((z-9.6/2)/2.4)**2 - 1"
# shape = "star"
nanopillars_prev = [0,0,0]
for f in range(6,12):#len(folder_list)):# in os.listdir(parent_path)
    folder = folder_list[f]
    if folder == "interp":
        continue
    results_path = f"{parent_path}/{folder}"
    for subfolder in os.listdir(mesh_folder):#f"{parent_path}/{folder}"):
        # if "mesh" in subfolder:
        #     mesh_file = f"{parent_path}/{folder}/{subfolder}/spreadCell_mesh.h5"
        # elif "results" in subfolder:
        #     results_path = f"{parent_path}/{folder}/{subfolder}"
        if subfolder in folder:
            mesh_file = f"{mesh_folder}/{subfolder}/spreadCell_mesh.h5"
    interp_path = f"{parent_path}/interp/{folder}_interp"
    # mesh_file = "/root/scratch/smart-comp-sci-data/mechanotransduction/fig-sweep-redo/0/"
    # results_path = "/root/shared/gitrepos/smart-comp-sci/mechanotransduction-example/results"
    # interp_path = "/root/shared/gitrepos/smart-comp-sci/mechanotransduction-example/interp"

    if mesh_file == "":
        messagebox.showinfo(title="Load mesh file.",
                            message="Please select the mesh file.")
        mesh_file = filedialog.askopenfilename()
    parent_mesh = smart.mesh.ParentMesh(
        mesh_filename=mesh_file,
        mesh_filetype="hdf5",
        name="parent_mesh",)

    comm = parent_mesh.mpi_comm
    dmesh = parent_mesh.dolfin_mesh
    dim = parent_mesh.dimensionality
    mf_cell = d.MeshFunction("size_t", dmesh, dim, value=0)
    mf_facet = d.MeshFunction("size_t", dmesh, dim-1, value=0)
    hdf5 = d.HDF5File(comm, mesh_file, "r")
    hdf5.read(mf_cell, f"/mf{dim}")
    hdf5.read(mf_facet, f"/mf{dim-1}")
    hdf5.close()

    # if shape == "circle":
    #     theta_expr = ""
    # elif shape == "star":
    #     theta_expr = "0.98 + 0.2814*cos(5*theta)"
    # elif shape == "rect":
    #     theta_expr = "rect0.6"

    # new_mesh, facet_markers_new, cell_markers_new = mesh_gen.create_3dcell(
    #         outerExpr=outerExpr13,
    #         innerExpr=innerExpr13,
    #         hEdge=0.5,
    #         hInnerEdge=0.5,
    #         thetaExpr=theta_expr,
    #     )
    nanopillar_radius = re.findall("_r.*_cell", folder)
    nanopillar_radius = float(nanopillar_radius[0][2:-5])
    nanopillar_height = re.findall("_h.*_p", folder)
    nanopillar_height = float(nanopillar_height[0][2:-2])
    nanopillar_spacing = re.findall("_p.*_r", folder)
    nanopillar_spacing = float(nanopillar_spacing[0][2:-2])
    contact_rad = re.findall("_cellRad.*_np", folder)
    if len(contact_rad) == 1:
        contact_rad = float(contact_rad[0][8:-3])
    else:
        contact_rad = re.findall("_cellRad.*$", folder)
        contact_rad = float(contact_rad[0][8:])
    nanopillars = [nanopillar_radius, nanopillar_height, nanopillar_spacing]
    if nanopillars!=nanopillars_prev:
        cell_mesh_new, facet_markers_new, cell_markers_new, substrate = mesh_gen.create_3dcell(contactRad=contact_rad,
                                                                    hEdge=1, hInnerEdge=1,
                                                                    thetaExpr="",
                                                                    nanopillars=nanopillars)
    nanopillars_prev = nanopillars

    if results_path == "":
        messagebox.showinfo(title="Load results directory",
                            message="Please select the results directory")
        results_path = filedialog.askdirectory()

    results_file_list = []
    tVec = []
    for file in os.listdir(results_path):
        # check the files which are end with specific extension
        if file.endswith(".h5"):
            results_file_list.append(file)
        if ind_files:
            if file.endswith(".xdmf"):
                xdmf_file = open(f"{results_path}/{file}", "r")
                xdmf_string = xdmf_file.read()
                found_pattern = re.findall(r"Time Value=\"?[^\s]+", xdmf_string)
                for i in range(len(found_pattern)):
                    tVec.append(float(found_pattern[i][12:-1]))
        else:
            if file.endswith(".xdmf") and tVec==[]:
                xdmf_file = open(f"{results_path}/{file}", "r")
                xdmf_string = xdmf_file.read()
                found_pattern = re.findall(r"Time Value=\"?[^\s]+", xdmf_string)
                for i in range(len(found_pattern)):
                    tVec.append(float(found_pattern[i][12:-1]))

    if ind_files:
        tVec = np.array(tVec)
        tVec = np.unique(tVec)
        tVec = np.sort(tVec)
        tVec = list(tVec)

    cell_vals = np.unique(mf_cell.array())
    cell_vals = cell_vals[np.logical_and(cell_vals !=0, cell_vals < 1e9)]
    facet_vals = np.unique(mf_facet.array())
    facet_vals = facet_vals[np.logical_and(facet_vals !=0, facet_vals < 1e9)]
    child_meshes = []
    child_mesh_len = []
    for i in range(len(cell_vals)+len(facet_vals)):
        if i < len(cell_vals):
            mesh = d.MeshView.create(mf_cell, cell_vals[i])
        else:
            mesh = d.MeshView.create(mf_facet, facet_vals[i-len(cell_vals)])
        child_meshes.append(mesh)
        child_mesh_len.append(len(mesh.coordinates()))

    for j in range(len(results_file_list)):
        cur_file = h5py.File(f"{results_path}/{results_file_list[j]}", "r")
        try:
            test_array = cur_file["VisualisationVector"]["0"][:]
        except:
            print("error")
            continue
        find_mesh = len(test_array) == np.array(child_mesh_len)
        if len(np.nonzero(find_mesh)[0]) != 1:
            ValueError("Could not identify submesh")
        else:
            submesh_idx = np.nonzero(find_mesh)[0][0]
            cur_mesh = child_meshes[submesh_idx]
            if submesh_idx < len(cell_vals):
                marker_cur = cell_vals[submesh_idx]
            else:
                marker_cur = facet_vals[submesh_idx-len(cell_vals)]
        
        subdomain_mf = d.MeshFunction("size_t", cur_mesh, cur_mesh.topology().dim(), 1)
        
        dx_cur = d.Measure("dx", domain=cur_mesh, subdomain_data=subdomain_mf)
        Vcur = d.FunctionSpace(cur_mesh, "P", 1)
        dvec = d.Function(Vcur)
        time_pt_tags = cur_file["VisualisationVector"].keys()
        num_time_points = len(time_pt_tags)
        dof_map = d.dof_to_vertex_map(Vcur)[:]
        
        if marker_cur in facet_markers_new.array():
            cur_mesh_new = d.MeshView.create(facet_markers_new, marker_cur)
        else:
            cur_mesh_new = d.MeshView.create(cell_markers_new, marker_cur)
        Vnew = d.FunctionSpace(cur_mesh_new, "P", 1)
        dvec_new = d.Function(Vnew)
        results_cur = d.XDMFFile(
            comm, str(f"{interp_path}/{results_file_list[j][0:-3]}_interp.xdmf")
        )
        results_cur.parameters["flush_output"] = True

        for i in range(num_time_points):
            try:
                cur_array = cur_file["VisualisationVector"][str(i)][:]
            except:
                print('error')
                continue
            # array matches mesh ordering; reorder according to dof mapping for Vcur
            cur_array = cur_array[dof_map]
            orig_coords = Vcur.tabulate_dof_coordinates()

            dvec.vector().set_local(cur_array)
            dvec.vector().apply("insert")
            dvec.set_allow_extrapolation(True)
            new_array = []

            # now interpolate onto new mesh
            new_coords = Vnew.tabulate_dof_coordinates()
            for k in range(len(new_coords)):
                if axisymm:
                    r_cur = np.sqrt(new_coords[k][0]**2 + new_coords[k][1]**2)
                    z_cur = new_coords[k][2]
                    new_array.append(dvec(r_cur, 0, z_cur))
                elif symm == 0.5:
                    x_cur, y_cur, z_cur = new_coords[k]
                    if y_cur < 0:
                        y_cur = -y_cur
                    new_array.append(dvec(x_cur, y_cur, z_cur))
                elif symm == 0.25:
                    x_cur, y_cur, z_cur = new_coords[k]
                    if y_cur < 0:
                        y_cur = -y_cur
                    if x_cur < 0:
                        x_cur = -x_cur
                    new_array.append(dvec(x_cur, y_cur, z_cur))
                elif symm == 0.125:
                    x_cur, y_cur, z_cur = new_coords[k]
                    if y_cur < 0:
                        y_cur = -y_cur
                    if x_cur < 0:
                        x_cur = -x_cur
                    if np.arctan2(y_cur, x_cur) > np.pi/4:
                        x_temp = x_cur
                        x_cur = y_cur
                        y_cur = x_temp
                    new_array.append(dvec(x_cur, y_cur, z_cur))
                else:
                    x_cur, y_cur, z_cur = new_coords[k]
                    new_array.append(dvec(x_cur, y_cur, z_cur))
            
            dvec_new.vector().set_local(new_array)
            dvec_new.vector().apply("insert")
            results_cur.write(dvec_new, tVec[i])
            print(f"Done with time step {i} for file {j}")